In [ ]:
# Load the data
with open('../../data/TinyStoriesV2-GPT4-train.txt', 'r', encoding='utf-8') as f:
    texts = f.readlines()

# Take a small subset for testing
texts = texts[:1000]  # Adjust this number based on your needs

print(f"Number of texts: {len(texts)}")
print(f"Sample text:\n{texts[0]}")


In [ ]:
from collections import defaultdict

def count_occurences(ufts):
    """Count all adjacent pairs in the list of tokens"""
    counts = defaultdict(int)
    for uft in ufts:
        # Convert string to list of characters if it's not already a list
        tokens = list(uft) if isinstance(uft, str) else uft
        for i in range(len(tokens) - 1):
            pair = (tokens[i], tokens[i + 1])
            counts[pair] += 1
    return counts

def merge_pair(ufts, pair, merged_token):
    """Replace all occurrences of pair with merged_token"""
    new_ufts = []
    for uft in ufts:
        # Convert string to list of characters if it's not already a list
        tokens = list(uft) if isinstance(uft, str) else uft
        i = 0
        new_tokens = []
        while i < len(tokens):
            if i < len(tokens) - 1 and (tokens[i], tokens[i + 1]) == pair:
                new_tokens.append(merged_token)
                i += 2
            else:
                new_tokens.append(tokens[i])
                i += 1
        new_ufts.append(new_tokens)
    return new_ufts

# Initialize vocabulary with basic characters
init_vocab = set(char for text in texts for char in text)
vocab_size = 10000
merges = []
utfs = [[c for c in text] for text in texts]  # Convert texts to list of characters

# Perform merges until desired vocabulary size is reached
for i in range(vocab_size - len(init_vocab)):
    # Count pair frequencies
    counts = count_occurences(utfs)
    if not counts:  # No more pairs to merge
        break
        
    # Find most frequent pair
    most_freq_pair = max(counts.items(), key=lambda x: x[1])[0]
    
    # Create new token for the pair
    new_token = ''.join(most_freq_pair)
    
    # Add merge operation to list
    merges.append((most_freq_pair, new_token))
    
    # Apply the merge throughout the dataset
    utfs = merge_pair(utfs, most_freq_pair, new_token)

# Build final vocabulary
vocab = list(init_vocab)
for _, new_token in merges:
    vocab.append(new_token)

print(f"Final vocabulary size: {len(vocab)}")
print(f"Number of merges performed: {len(merges)}")


![](../../images/unicode2_prob.png)

### (a) 

#### It is about the effectivness of memory. In UTF32 all the symbols contain 4 bytes, while the most popular symbols in UTF8 would consist of one byte. This reduces the amount of memory used. Btw, UTF32 is more effective when it comes to searching N's symbol. It is O(N) in UTF8, since we don't know how many bytes each symbol contains. But in UTF32 it is all statically structured. 

### (b)

#### The answer comes from (a), since some symbols, for instance, emojies can constist of more than one byte. This contradicts the implementation of the function, since it always breaks the symbols into 1 byte.

### (c) 

#### In UTF8 every first byte is an "index-byte" that encodes the range of bytes that the symbol is constists of (1-4). If we don't put this index-byte into the beginning of the sequence, the decoding is impossible

In [5]:
import regex as re
PAT = r"""'(?:[sdmt]|ll|ve|re)| ?\p{L}+| ?\p{N}+| ?[^\s\p{L}\p{N}]+|\s+(?!\S)|\s+"""
re.findall(PAT, "some text that i'll pre-tokenize")

['some', ' text', ' that', ' i', "'ll", ' pre', '-', 'tokenize']

![](../../images/unicode2_exampt.png)

In [6]:
with open("/home/ed/work/other_projects/stanLLMasingments/StanfP1/data/TinyStoriesV2-GPT4-train.txt", 'r', encoding='utf-8') as f:
    text = f.read()

In [9]:
reduced_text = text[:10000000]

In [16]:
regexed = re.findall(PAT, reduced_text)

In [17]:
regexed[:15]

['\n',
 'Once',
 ' upon',
 ' a',
 ' time',
 ' there',
 ' was',
 ' a',
 ' little',
 ' boy',
 ' named',
 ' Ben',
 '.',
 ' Ben',
 ' loved']

In [34]:
# init_vocab = ['<|endoftext|>'] + [bytes([i]).decode('utf-8') for i in range(256)]
init_vocab = ['<|endoftext|>'] + [chr(i) for i in range(256)]

In [44]:
utfs = [i.encode('utf-8') for i in regexed]
utfs


[b'\n',
 b'Once',
 b' upon',
 b' a',
 b' time',
 b' there',
 b' was',
 b' a',
 b' little',
 b' boy',
 b' named',
 b' Ben',
 b'.',
 b' Ben',
 b' loved',
 b' to',
 b' explore',
 b' the',
 b' world',
 b' around',
 b' him',
 b'.',
 b' He',
 b' saw',
 b' many',
 b' amazing',
 b' things',
 b',',
 b' like',
 b' beautiful',
 b' vases',
 b' that',
 b' were',
 b' on',
 b' display',
 b' in',
 b' a',
 b' store',
 b'.',
 b' One',
 b' day',
 b',',
 b' Ben',
 b' was',
 b' walking',
 b' through',
 b' the',
 b' store',
 b' when',
 b' he',
 b' came',
 b' across',
 b' a',
 b' very',
 b' special',
 b' vase',
 b'.',
 b' When',
 b' Ben',
 b' saw',
 b' it',
 b' he',
 b' was',
 b' amazed',
 b'!',
 b'  ',
 b'\n',
 b'He',
 b' said',
 b',',
 b' \xe2\x80\x9c',
 b'Wow',
 b',',
 b' that',
 b' is',
 b' a',
 b' really',
 b' amazing',
 b' vase',
 b'!',
 b' Can',
 b' I',
 b' buy',
 b' it',
 b'?\xe2\x80\x9d',
 b' ',
 b'\n',
 b'The',
 b' shopkeeper',
 b' smiled',
 b' and',
 b' said',
 b',',
 b' \xe2\x80\x9c',
 b'Of',
 b'

In [48]:
from collections import defaultdict

def count_occurences(ufts):
    counts = defaultdict(int)
    for tok in utfs:                       # utfs — список bytes
        for i in range(len(tok) - 1):
            pair = tok[i : i + 2]        
            counts[pair] += 1
    return counts
    
    

In [50]:
def merge_pair(utfs, pair_to_merge): 
    new_pair = ''.join(pair_to_merge)
    n = len(utfs)
    new_utfs = []
    for word in utfs:
        new_token = []
        i = 0
        while i < n:
            if i < n - 1 and (word[i], word[i+1]) == pair_to_merge:
                new_token.append(new_pair)
                i += 2
            else:
                new_token.append(word[i])
                i += 1
        new_utfs.append(new_token)
    return new_utfs

In [ ]:
from collections import defaultdict
vocab_size = 10000
merges = []
counts = defaultdict(int)
for i in range(vocab_size - len(init_vocab)):
    counts = count_occurences(ufts=utfs)
    most_prob = max(counts, key=counts.get)
    merges.append
    ufts = [tok.replace(most_prob, )]



defaultdict(int,
            {b'On': 18801,
             b'nc': 11926,
             b'ce': 19239,
             b' u': 19598,
             b'up': 14207,
             b'po': 15619,
             b'on': 41441,
             b' a': 213320,
             b' t': 285872,
             b'ti': 21999,
             b'im': 39838,
             b'me': 49081,
             b'th': 195722,
             b'he': 285524,
             b'er': 100455,
             b're': 77829,
             b' w': 141046,
             b'wa': 78714,
             b'as': 64452,
             b' l': 59115,
             b'li': 32966,
             b'it': 65931,
             b'tt': 17914,
             b'tl': 12333,
             b'le': 44476,
             b' b': 98948,
             b'bo': 18055,
             b'oy': 12676,
             b' n': 40958,
             b'na': 14860,
             b'am': 27311,
             b'ed': 117007,
             b' B': 12161,
             b'Be': 5522,
             b'en': 70022,
             b'lo': 33335,
     